
# Train Notebook — EG Property Estimator

This notebook walks through the **machine learning flow** used in this project:
1. Load and inspect the dataset (CSV).
2. Clean & prepare the data.
3. Feature engineering (turn raw columns into stronger signals).
4. Build a modeling pipeline (preprocessing + model).
5. Cross-validation and a **baseline blend** for stability.
6. City-level calibration (bias correction).
7. Final evaluation + simple visualizations.
8. Save the trained model (`.joblib`) + artifacts for serving with **FastAPI**.

> **Inputs:** a cleaned CSV of listing data (`Eg_RealState_Data_Cleaned.csv`) placed at `ml-layer/data/`  
> **Outputs:** a trained model (`ppm2_model.joblib`), baseline median tables, and a city-bias file, all saved to `ml-layer/artifacts/` and consumed by the API.



## How to run this notebook

- If you don't have Jupyter:
  ```bash
  # (activate your venv first)
  pip install notebook  # or: pip install jupyterlab
  jupyter notebook       # or: jupyter lab
  ```
- Open **`train_notebook.ipynb`** and run cells top-to-bottom.
- Make sure your dataset is at: `ml-layer/data/Eg_RealState_Data_Cleaned.csv`.


In [ ]:

import os
from pathlib import Path
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.ensemble import HistGradientBoostingRegressor

import joblib

# Optional: MLflow tracking (uncomment if you'd like to log runs)
# import mlflow
# import mlflow.sklearn

# Paths
PROJECT_ROOT = Path.cwd()  # adjust if running from a different folder
DATA_PATH = PROJECT_ROOT / "ml-layer" / "data" / "Eg_RealState_Data_Cleaned.csv"
ARTIFACT_DIR = PROJECT_ROOT / "ml-layer" / "artifacts"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

print('Data path:', DATA_PATH)
print('Artifacts will be saved to:', ARTIFACT_DIR)



## 1. Load and quickly inspect the data

We expect at least these columns:
- `price`, `area`, `bedrooms`, `bathrooms`, `type`, `city`, `region`
- (optional) `level`, `furnished`, `rent`

We'll load the CSV and do a tiny sanity check.


In [ ]:

df = pd.read_csv(DATA_PATH)
print('Raw shape:', df.shape)
print('Columns:', list(df.columns))
df.head(3)


In [ ]:

# Basic info & nulls
display(df.describe(include='all').T.head(20))
df.isna().mean().sort_values(ascending=False).head(20)



## 2. Cleaning & Feature Engineering (high level)

**Cleaning**
- Keep **sale** listings only (if `rent` exists, filter where `rent == "no"`).
- Drop obviously bad city tokens (`""`, `"nan"`, `"none"`, `"null"`, `"v"`).
- Bucket **rare** regions into a single `"__other__"` bucket.
- Remove **outliers** by trimming **price-per-m² (ppm²)**:
  - Global trim (1–99th percentiles)
  - Per-city robust trim (5–95th) if the city has enough data (≥50 rows)

**Feature Engineering**
- Cast numeric-like columns to numbers; coerce unknowns to NaN.
- `level_num` (floor parsed from string), `is_ground` (1 if ground).
- `furnished_bin` (1 for "yes", 0 for "no").
- Ratios & transforms: `area_per_bedroom`, `bathrooms_per_bedroom`, `rooms_total`, `bed_bath_ratio`, `log_area`, `sqrt_area`, `area_per_room`.


In [ ]:

def normalize_and_engineer(df):
    df = df.copy()

    # ensure presence of key cols
    for c in ["type","city","region","level","furnished","bedrooms","bathrooms","area","price"]:
        if c not in df.columns:
            df[c] = np.nan

    # standardize strings
    for c in ["type","city","region","level","furnished"]:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip().str.lower()

    # coerce numbers
    for c in ["area","bedrooms","bathrooms","price"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # level -> numeric + ground flag
    def level_to_num(s):
        if isinstance(s, str):
            if "ground" in s:
                return 0.0
            try:
                return float(s)
            except:
                return np.nan
        try:
            return float(s)
        except:
            return np.nan

    df["level_num"] = df["level"].apply(level_to_num)
    df["is_ground"] = (df["level"].fillna("").str.contains("ground")).astype(float)

    # furnished -> binary
    df["furnished_bin"] = df["furnished"].map({"yes":1.0, "no":0.0}).fillna(0.0).astype(float)

    # engineered numeric
    safe_bed = df["bedrooms"].replace(0, np.nan)
    safe_bath = df["bathrooms"].replace(0, np.nan)
    safe_area = df["area"].clip(lower=1)

    df["area_per_bedroom"]     = df["area"] / safe_bed
    df["bathrooms_per_bedroom"] = df["bathrooms"] / safe_bed
    df["rooms_total"]          = df["bedrooms"] + df["bathrooms"]
    df["log_area"]             = np.log1p(safe_area)
    df["sqrt_area"]            = np.sqrt(safe_area)
    df["bed_bath_ratio"]       = df["bedrooms"] / safe_bath
    df["area_per_room"]        = df["area"] / df["rooms_total"]

    return df


In [ ]:

# 2a) Keep sales only if 'rent' exists
if "rent" in df.columns:
    df["rent"] = df["rent"].astype(str).str.lower()
    df = df[df["rent"] == "no"].copy()

# 2b) Drop rows missing key fields
required = ["price","area","bedrooms","bathrooms","type","city","region"]
df = df.dropna(subset=[c for c in required if c in df.columns]).copy()

# 2c) Feature engineering
df = normalize_and_engineer(df)

# 2d) Global ppm² trimming
df["ppm2_tmp"] = df["price"] / df["area"].clip(lower=1)
lo, hi = df["ppm2_tmp"].quantile([0.01, 0.99])
df = df[(df["ppm2_tmp"] >= lo) & (df["ppm2_tmp"] <= hi)].copy()

# 2e) Remove bad city tokens
bad_cities = {"v","", "nan","none","null"}
df = df[~df["city"].isin(bad_cities)].copy()

# 2f) Bucket rare regions
vc = df["region"].value_counts()
keep_regions = set(vc[vc >= 30].index)
df["region"] = df["region"].where(df["region"].isin(keep_regions), "__other__")

# 2g) Per-city robust trimming (5-95) for cities with adequate data (>=50)
city_counts = df.groupby("city")["ppm2_tmp"].transform("count")
city_lo = df.groupby("city")["ppm2_tmp"].transform(lambda s: s.quantile(0.05))
city_hi = df.groupby("city")["ppm2_tmp"].transform(lambda s: s.quantile(0.95))
use_city = city_counts >= 50
df = df[(~use_city & (df["ppm2_tmp"] >= lo) & (df["ppm2_tmp"] <= hi)) |
        (use_city  & (df["ppm2_tmp"] >= city_lo) & (df["ppm2_tmp"] <= city_hi))].copy()

df = df.drop(columns=["ppm2_tmp"])
print('Post-clean shape:', df.shape)
df.head(3)



## 3. Category encoding

- `type` is **low-cardinality** → One-Hot Encoding  
- `city` and `region` are **high-cardinality** → **Smoothed Target Encoding (TE)**

The TE encodes each category to a smoothed mean of the target (ppm²), learned **inside the training folds** to avoid leakage.


In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin

class SmoothedTargetEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, cols, alpha=10.0):
        self.cols = cols
        self.alpha = alpha
        self.prior_ = None
        self.maps_ = {}

    def fit(self, X, y=None):
        if y is None:
            raise ValueError("SmoothedTargetEncoder requires y during fit.")
        X = X.copy()
        self.prior_ = float(np.mean(y))
        self.maps_ = {}
        for c in self.cols:
            s = pd.Series(X[c].astype(str).values, name=c)
            dfc = pd.DataFrame({c: s, "y": y})
            grp = dfc.groupby(c)["y"].agg(["sum","count"])
            enc = (grp["sum"] + self.prior_ * self.alpha) / (grp["count"] + self.alpha)
            self.maps_[c] = enc.to_dict()
        return self

    def transform(self, X):
        X = X.copy()
        out = []
        for c in self.cols:
            m = self.maps_.get(c, {})
            out.append(X[c].astype(str).map(m).fillna(self.prior_).to_numpy().reshape(-1,1))
        return np.concatenate(out, axis=1)

    def get_feature_names_out(self, input_features=None):
        return np.array([f"te_{c}" for c in self.cols])



## 4. A robust baseline (medians)

We compute **median ppm²** at progressively broader levels:
1. `(city, region, type)`
2. `(city, region)`
3. `(city)`
4. Global median

At prediction time we try level 1; if missing, fall back to 2 → 3 → global.


In [ ]:

def fit_ppm2_baseline(X: pd.DataFrame, yppm2: pd.Series):
    dfb = pd.DataFrame({
        "city": X["city"].astype(str),
        "region": X["region"].astype(str),
        "type": X["type"].astype(str),
        "ppm2": yppm2.astype(float)
    })
    m_crt = dfb.groupby(["city","region","type"])["ppm2"].median().rename("m_crt").reset_index()
    m_cr  = dfb.groupby(["city","region"])["ppm2"].median().rename("m_cr").reset_index()
    m_c   = dfb.groupby(["city"])["ppm2"].median().rename("m_c").reset_index()
    m_g   = float(dfb["ppm2"].median())
    return {"m_crt": m_crt, "m_cr": m_cr, "m_c": m_c, "m_g": m_g}

def predict_ppm2_baseline(X: pd.DataFrame, stats: dict) -> np.ndarray:
    s = X[["city","region","type"]].astype(str).copy()
    out = s.merge(stats["m_crt"], on=["city","region","type"], how="left")
    out = out.merge(stats["m_cr"], on=["city","region"], how="left")
    out["ppm2"] = out["m_crt"].fillna(out["m_cr"])
    out = out.merge(stats["m_c"], on=["city"], how="left")
    out["ppm2"] = out["ppm2"].fillna(out["m_c"]).fillna(stats["m_g"])
    return out["ppm2"].to_numpy()



## 5. Train/Validation split and pipeline

- **Target**: `ppm² = price / area`
- **Pipeline**:
  - Numeric features: passthrough
  - `type`: One-Hot Encoding
  - `city`, `region`: Smoothed Target Encoding
  - Regressor: HistGradientBoostingRegressor (with `loss="absolute_error"`)
  - Wrap in `TransformedTargetRegressor` to learn on **log1p(ppm²)** for stability


In [ ]:

# Select columns
num_feats_num = [
    "area","bedrooms","bathrooms",
    "level_num","is_ground","furnished_bin",
    "area_per_bedroom","bathrooms_per_bedroom","rooms_total",
    "log_area","sqrt_area","bed_bath_ratio","area_per_room",
]
cat_ohe_cols = ["type"]
cat_te_cols  = ["city","region"]

cols_needed = ["price"] + num_feats_num + cat_ohe_cols + cat_te_cols
dfm = df[cols_needed].dropna().copy()

# Base arrays
X = dfm.drop(columns=["price"])
y_price = dfm["price"].astype(float)
area_clip = X["area"].clip(lower=1).astype(float)
y_ppm2 = (y_price / area_clip).astype(float)

# Split
X_train, X_test, yppm2_train, yppm2_test, yprice_train, yprice_test, area_train, area_test = train_test_split(
    X, y_ppm2, y_price, area_clip, test_size=0.15, random_state=42, stratify=X["city"]
)

# Preprocessor + model
pre = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_feats_num),
        ("type_ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_ohe_cols),
        ("cat_te", SmoothedTargetEncoder(cols=cat_te_cols, alpha=10.0), cat_te_cols),
    ]
)

reg = HistGradientBoostingRegressor(
    loss="absolute_error",
    max_depth=None,
    learning_rate=0.06,
    max_iter=900,
    l2_regularization=0.0,
    early_stopping=True,
    random_state=42,
)

model = Pipeline(steps=[
    ("pre", pre),
    ("reg", TransformedTargetRegressor(regressor=reg, func=np.log1p, inverse_func=np.expm1)),
])

model



## 6. Cross-validation & Blending

We blend the model ppm² with the median-baseline ppm²:

\[ ppm²\_blend = \alpha \cdot ppm²\_{model} + (1-\alpha) \cdot ppm²\_{baseline} \]

We pick **α** by 5-fold cross-validation on the training split.


In [ ]:

ALPHA_GRID = [0.5, 0.6, 0.7, 0.8]
kf = KFold(n_splits=5, shuffle=True, random_state=42)
alpha_rmse = {a: [] for a in ALPHA_GRID}

for tr_idx, va_idx in kf.split(X_train):
    X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    yppm2_tr, yppm2_va = yppm2_train.iloc[tr_idx], yppm2_train.iloc[va_idx]
    yprice_va = yprice_train.iloc[va_idx]
    area_va = area_train.iloc[va_idx]

    model.fit(X_tr, yppm2_tr)
    pred_ppm2_va_model = model.predict(X_va)

    stats = fit_ppm2_baseline(X_tr, yppm2_tr)
    pred_ppm2_va_base = predict_ppm2_baseline(X_va, stats)

    for a in ALPHA_GRID:
        pred_price_va = (a * pred_ppm2_va_model + (1 - a) * pred_ppm2_va_base) * area_va.values
        rmse = root_mean_squared_error(yprice_va, pred_price_va)
        alpha_rmse[a].append(rmse)

best_alpha = min(ALPHA_GRID, key=lambda a: np.mean(alpha_rmse[a]))
cv_rmse_mean = float(np.mean(alpha_rmse[best_alpha]))
cv_rmse_std  = float(np.std(alpha_rmse[best_alpha]))

print("Best alpha:", best_alpha)
print(f"CV RMSE mean={cv_rmse_mean:,.0f} (±{cv_rmse_std:,.0f})")



## 7. City-level bias calibration (OOF)

We compute a multiplicative correction per city using **out-of-fold** predictions:
\[ bias(city) = \mathrm{median}\_{OOF} \left( \frac{y\_{price}}{\hat{y}\_{price}} \right) \]
We clip it to [0.6, 1.6] for stability and apply it after blending.


In [ ]:

city_ratios = {}
for tr_idx, va_idx in kf.split(X_train):
    X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    yppm2_tr, yppm2_va = yppm2_train.iloc[tr_idx], yppm2_train.iloc[va_idx]
    yprice_va = yprice_train.iloc[va_idx]
    area_va = area_train.iloc[va_idx]

    model.fit(X_tr, yppm2_tr)
    pred_ppm2_va_model = model.predict(X_va)
    stats = fit_ppm2_baseline(X_tr, yppm2_tr)
    pred_ppm2_va_base = predict_ppm2_baseline(X_va, stats)

    pred_price_va = (best_alpha * pred_ppm2_va_model + (1 - best_alpha) * pred_ppm2_va_base) * area_va.values
    ratio = (yprice_va / np.maximum(pred_price_va, 1.0)).to_numpy()

    for city, r in zip(X_va["city"].astype(str).to_numpy(), ratio):
        city_ratios.setdefault(city, []).append(r)

city_bias = {c: float(np.clip(np.median(rs), 0.6, 1.6)) for c, rs in city_ratios.items()}
len(city_bias), list(city_bias.items())[:5]



## 8. Final fit & evaluation


In [ ]:

# Fit on full training set
model.fit(X_train, yppm2_train)

# Test predictions
pred_ppm2_test_model = model.predict(X_test)
stats_full = fit_ppm2_baseline(X_train, yppm2_train)
pred_ppm2_test_base = predict_ppm2_baseline(X_test, stats_full)

pred_price_test = (best_alpha * pred_ppm2_test_model + (1 - best_alpha) * pred_ppm2_test_base) * area_test.values
bias_vec = np.array([city_bias.get(c, 1.0) for c in X_test["city"].astype(str).to_numpy()])
pred_price_test_adj = pred_price_test * bias_vec

def eval_metrics(y_true_price, pred_price):
    rmse = root_mean_squared_error(y_true_price, pred_price)
    mae  = mean_absolute_error(y_true_price, pred_price)
    mape = (np.abs((y_true_price - pred_price) / np.maximum(y_true_price, 1.0))).mean() * 100
    return rmse, mae, mape

rmse_raw, mae_raw, mape_raw = eval_metrics(yprice_test, pred_price_test)
rmse, mae, mape = eval_metrics(yprice_test, pred_price_test_adj)

print(f"Pre-bias Test RMSE={rmse_raw:,.0f} | MAE={mae_raw:,.0f} | MAPE={mape_raw:.2f}%")
print(f"Post-bias Test RMSE={rmse:,.0f} | MAE={mae:,.0f} | MAPE={mape:.2f}%")



## 9. Visualizations

We’ll plot:
- **Predicted vs Actual** (price EGP)
- **Residuals histogram**
- **Top-10 cities by MAPE** (bar chart)


In [ ]:

# Predicted vs Actual
plt.figure()
plt.scatter(yprice_test, pred_price_test_adj, alpha=0.4)
mx = float(max(yprice_test.max(), pred_price_test_adj.max()))
plt.plot([0, mx], [0, mx])
plt.xlabel("Actual Price (EGP)")
plt.ylabel("Predicted Price (EGP)")
plt.title("Predicted vs Actual (Test)")
plt.show()


In [ ]:

# Residuals histogram
resid = yprice_test - pred_price_test_adj
plt.figure()
plt.hist(resid, bins=50)
plt.xlabel("Residual (Actual - Predicted) [EGP]")
plt.ylabel("Count")
plt.title("Residuals Histogram (Test)")
plt.show()


In [ ]:

# Worst cities by MAPE (top 10)
dfm_plot = pd.DataFrame({
    "city": X_test["city"].values,
    "y": yprice_test.values,
    "p": pred_price_test_adj
})

def agg(g):
    e = g["y"] - g["p"]
    mae = np.mean(np.abs(e))
    rmse = np.sqrt(np.mean(e ** 2))
    mape = np.mean(np.abs(e) / np.maximum(g["y"], 1.0)) * 100
    return pd.Series({"n": len(g), "MAE": mae, "RMSE": rmse, "MAPE": mape})

out = dfm_plot.groupby("city").apply(agg).sort_values("MAPE", ascending=False).head(10)

plt.figure()
plt.bar(out.index.astype(str), out["MAPE"].values)
plt.xticks(rotation=45, ha="right")
plt.ylabel("MAPE (%)")
plt.title("Top-10 Cities by MAPE (Test)")
plt.tight_layout()
plt.show()

out.round(1)



## 10. Save artifacts for serving

We save:
- **Model** (`ppm2_model.joblib`): the sklearn Pipeline that predicts **ppm²**
- **Baseline medians**: `m_crt.csv`, `m_cr.csv`, `m_c.csv`, and `m_g.txt`
- **City bias**: `city_bias.csv`

> The FastAPI app loads these once at startup and exposes **`POST /predict`** for the frontend.


In [ ]:

# Save model
model_path = ARTIFACT_DIR / "ppm2_model.joblib"
joblib.dump(model, model_path)

# Save baseline stats
stats_dir = ARTIFACT_DIR / "baseline_stats"
stats_dir.mkdir(exist_ok=True, parents=True)
stats_full["m_crt"].to_csv(stats_dir / "m_crt.csv", index=False)
stats_full["m_cr"].to_csv(stats_dir / "m_cr.csv", index=False)
stats_full["m_c"].to_csv(stats_dir / "m_c.csv", index=False)
(stats_dir / "m_g.txt").write_text(str(stats_full["m_g"]))

# Save city bias
pd.Series(city_bias, name="bias").to_csv(ARTIFACT_DIR / "city_bias.csv")

print("Saved:", model_path)
print("Saved baseline stats to:", stats_dir)
print("Saved city_bias.csv to:", ARTIFACT_DIR / "city_bias.csv")



## Appendix — Where this logic lives in the repo

- `ml-layer/src/utils.py` — contains `normalize_and_engineer` for feature engineering (mirrored here).
- `ml-layer/src/encoders.py` — contains `SmoothedTargetEncoder` (mirrored here).
- `ml-layer/src/predict.py` — loads `ppm2_model.joblib`, median stats, and city bias; exposes a class used by FastAPI (`PriceEstimator`).

In production, the **FastAPI** app imports `PriceEstimator` and exposes **`/predict`**, and the **React (Vite)** frontend posts form data to it and displays the result.
